In [1]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline  
import re
from data_preprocessing import *
from sklearn.model_selection import RandomizedSearchCV, train_test_split 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import eli5
from eli5.sklearn import PermutationImportance
import time
from sklearn.ensemble import GradientBoostingRegressor

### EDA can be found in https://github.com/yuxinchenNU/chronic_health/blob/master/exploratory_data_analysis_US.ipynb 
### Obtain cleaned data

In [2]:
feature_cols, df = preprocessing() # obtain cleaned data

### Prepare data for training and testing

In [3]:
# Choose target variable
y = df['% Fair/Poor'].values

In [4]:
# Choose features
# axis 1 refers to the columns
features= df[feature_cols]
# Remove the labels from the features
features = features.drop('% Fair/Poor', axis = 1)
# Remove features that I think will corrupt the model
features = features.drop('MV Mortality Rate', axis = 1)

In [5]:
features.columns.values

array(['% Children in Poverty', 'Teen Birth Rate',
       'Mentally Unhealthy Days', '% Some College', '% Smokers',
       '% Physically Inactive', '% LBW', '% Excessive Drinking',
       '% Single-Parent Households', '% Obese', '% Unemployed',
       '% Mammography', '% Uninsured', 'Chlamydia Rate'], dtype=object)

In [6]:
# Using Skicit-learn to split data into training and testing sets
# Split the data into training and testing sets
train_features, test_features, train_y, test_y = train_test_split(features, y, test_size = 0.25, random_state = 1)

## First model that I am looking at is random forest

In [7]:
# Instantiate model with 1000 decision trees
RF_REGRESSOR = RandomForestRegressor(n_estimators=1000, random_state=1, n_jobs=-1)

# Train the model on training data
RF_REGRESSOR.fit(train_features, train_y)

# Use the forest's predict method on the test data
predictions = RF_REGRESSOR.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_y)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_y)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('1-MAPE:', round(accuracy, 2), '%.')

Mean Absolute Error: 1.36 degrees.
1-MAPE: 91.51 %.


In [8]:

print('R squared score: ', r2_score(predictions, test_y))
print('Mean squared error: ', mean_squared_error(predictions, test_y))

R squared score:  0.80906288041269
Mean squared error:  3.70535110933345


## Report feature importances

In [9]:
def feature_importance_built_in(trained_model, train_data):
    """
    use scikit-learn's built in feature importance method
    
    Args:
        - (scikit-learn Estimator object): trained model
        - (Pandas DataFrame): training data
    Returns:
        - (list of Python tuples): (str describing feature, float describing feature importance) 
    """
    # Obtain feature importances
    importances = list(trained_model.feature_importances_)

    # Combine a list of tuples with feature variable and importance
    feature_importances = [(feature, round(importance, 4)) for feature, importance in zip(train_data.columns.values, importances)]

    # Sort feature importances by most important first
    feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)
    
    return feature_importances

feature_importances = feature_importance_built_in(RF_REGRESSOR, train_features)

# Print
[print('Feature Variable: {:40} Importance: {}'.format(*pair)) for pair in feature_importances];

Feature Variable: % Children in Poverty                    Importance: 0.4593
Feature Variable: Mentally Unhealthy Days                  Importance: 0.1817
Feature Variable: Teen Birth Rate                          Importance: 0.0944
Feature Variable: % Smokers                                Importance: 0.0378
Feature Variable: % Excessive Drinking                     Importance: 0.037
Feature Variable: % Physically Inactive                    Importance: 0.0344
Feature Variable: % Some College                           Importance: 0.0277
Feature Variable: % Uninsured                              Importance: 0.0221
Feature Variable: % Obese                                  Importance: 0.0206
Feature Variable: % Mammography                            Importance: 0.019
Feature Variable: Chlamydia Rate                           Importance: 0.0187
Feature Variable: % LBW                                    Importance: 0.017
Feature Variable: % Unemployed                             Importan

## Note: another way to compute feature importance: permutation importance

In [10]:

perm = PermutationImportance(RF_REGRESSOR, random_state=1).fit(train_features, train_y)
eli5.show_weights(perm, feature_names = train_features.columns.tolist())

Weight,Feature
0.2907 ± 0.0033,Mentally Unhealthy Days
0.2471 ± 0.0073,% Children in Poverty
0.1451 ± 0.0045,Teen Birth Rate
0.0639 ± 0.0013,% Excessive Drinking
0.0600 ± 0.0030,% Smokers
0.0482 ± 0.0015,% Physically Inactive
0.0460 ± 0.0014,% Some College
0.0319 ± 0.0011,% Uninsured
0.0298 ± 0.0009,Chlamydia Rate
0.0260 ± 0.0009,% Mammography


## Keep features that have feature importance larger than certain threshold

In [11]:
# extract features that have importance > a threshold
effective_features = [pair[0] for pair in feature_importances if pair[1] > 0.02]

# define effective feature variables
eff_train_features = train_features.loc[:, effective_features]
eff_test_features = test_features.loc[:, effective_features]

## Training on effective features

In [12]:
RF_REGRESSOR.fit(eff_train_features, train_y)

# Use the forest's predict method on the test data
eff_predictions = RF_REGRESSOR.predict(eff_test_features)

# Calculate the absolute errors
eff_errors = abs(eff_predictions - test_y)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(eff_errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
eff_mape = 100 * (eff_errors / test_y)

# Calculate and display accuracy
eff_accuracy = 100 - np.mean(eff_mape)
print('1-MAPE:', round(eff_accuracy, 2), '%.')

Mean Absolute Error: 1.43 degrees.
Accuracy: 91.04 %.


In [13]:
print('R squared score: ', r2_score(eff_predictions, test_y))
print('Mean squared error: ', mean_squared_error(eff_predictions, test_y))

R squared score:  0.7904627958326906
Mean squared error:  4.085570595351787


## Compare trade-offs
### Performance seem to decrease by a little bit, now test whether reducing the number of features reduces training time

In [14]:

# measure how long training with all features takes
start_time = time.time()
RF_REGRESSOR.fit(train_features, train_y) # training with all features
prediction_all = RF_REGRESSOR.predict(test_features) # prediction with all features
end_time = time.time()

running_time_all = end_time - start_time

# measure how long training with effective features takes
start_time = time.time()
RF_REGRESSOR.fit(eff_train_features, train_y) # training with all features
prediction_eff = RF_REGRESSOR.predict(eff_test_features) # prediction with all features
end_time = time.time()

running_time_eff = end_time - start_time

print('Training and testing with all features takes', round(running_time_all, 4), 'seconds.')
print('Training and testing with effective features takes', round(running_time_eff, 4), 'seconds.')

Training and testing with all features takes 39.4622 seconds.
Training and testing with effective features takes 25.014 seconds.


In [15]:
print('Relative decrease in accuracy is ', (accuracy - eff_accuracy)/accuracy)
print('Relative decrease in running time is ', (running_time_all - running_time_eff)/running_time_all)

Relative decrease in accuracy is  0.005133754784194948
Relative decrease in running time is  0.36612875034475423


## Hyperparameter tuning
### Random search 
(NOTE: because the only preprocessing is only removing NaNs and outliers, so I didn't do preprocessing separately in each fold in the k-fold cross validation, in the future if I need to do more preprocessing such as standardization or normalization, then I need to perform preprocessing separately in each fold and separately in training and test sets)

In [17]:
def hyperparameter_tuning_random_search(model, parameter_grid, train_features, train_y):
    """
    use Random Search to perform hyperparameter tuning
    
    Args:
        - (scikit-learn Estimator object): trained model 
        - (Python dictionary): lists of all parameter combinations to explore, labeled by key 
        - (Pandas dataframe): training features
        - (numpy array): training target variables
    
    Returns:   
        - (scikit-learn RandomizedSearchCV object): output of random search
    """
    # create the RandomizedSearchCV object
    random_search = RandomizedSearchCV(estimator=model, param_distributions=parameter_grid, n_iter=100, 
                                        cv=3, random_state=1, n_jobs=-1)  
    
    # fine-tune the hyperparameters
    random_search.fit(train_features, train_y)
    
    return random_search
    
# hyerparameters grid to search within
param_grid_rf = {'n_estimators': [100, 250, 400, 500, 600, 700, 1000],
         'max_features': ['auto', 'sqrt'],
         'max_depth': [2, 5, 10, 20, 30, 40, 50],
         'min_samples_split': [3, 5, 7, 10, 15],
         'min_samples_leaf': [1, 2, 4],
         'bootstrap': [True, False]}

# declare the regressor
RF_REGRESSOR = RandomForestRegressor()

# obtain the RandomizedSearchCV fitted object
random_search_rf = hyperparameter_tuning_random_search(RF_REGRESSOR, param_grid_rf, train_features, train_y)

# get the best model
final_model_rf = random_search_rf.best_estimator_

In [18]:
# obtain hyperparameters used in the best model
print(random_search_rf.best_params_)
print(random_search_rf.best_score_)

{'n_estimators': 1000, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': False}
0.8421467940604642


In [19]:
# evaluate the "best" model
def test_set_performance(model, test_features, test_y):
    """
    assess R-squared score and accuracy (1-MAPE) with given trained model, test data, test_y
    
    Args:
        - (scikit-learn Estimator object): trained model
        - (Pandas DataFrame): test data
        - (NumPy array): test target variables
    Returns:
        - (Python float): R-squared score 
        - (Python float): test set accuracy
    """
    prediction_model = model.predict(test_features)
    errors = abs(prediction_model - test_y)
    mape = 100*np.mean(errors/test_y)
    accuracy = 100 - mape
    r2 = r2_score(prediction_model, test_y)
    
    return accuracy, r2

accuracy_final_rf, r2_final_rf = test_set_performance(final_model_rf, test_features, test_y)
print('R squared score: ', r2_final_rf)
print('Accuracy:', round(accuracy_final_rf, 2), '%.')

R squared score:  0.8172780871807593
Accuracy: 91.89 %.


In [20]:
# create a baseline model
BASELINE_MODEL = RandomForestRegressor(n_estimators=5, random_state=1)
BASELINE_MODEL.fit(train_features, train_y)
prediction_baseline_model = BASELINE_MODEL.predict(test_features)
errors_baseline = abs(prediction_baseline_model - test_y)
mape_baseline = 100*np.mean(errors_baseline/test_y)
accuracy_baseline = 100 - mape_baseline
r2_baseline = r2_score(prediction_baseline_model, test_y)

print('Baseline R squared score: ', r2_baseline)
print('Baseline model 1-MAPE:', round(accuracy_baseline, 2), '%.')

Baseline R squared score:  0.7656896519291806
Baseline model accuracy: 90.31 %.


## Feature importance analysis on the "best" model

In [21]:
feature_importances = feature_importance_built_in(final_model_rf, eff_train_features)

# Print
[print('Feature Variable: {:40} Importance: {}'.format(*pair)) for pair in feature_importances];

Feature Variable: % Children in Poverty                    Importance: 0.1964
Feature Variable: Teen Birth Rate                          Importance: 0.1553
Feature Variable: Mentally Unhealthy Days                  Importance: 0.1397
Feature Variable: % Smokers                                Importance: 0.0935
Feature Variable: % Excessive Drinking                     Importance: 0.0731
Feature Variable: % Physically Inactive                    Importance: 0.0687
Feature Variable: % Uninsured                              Importance: 0.0584
Feature Variable: % Some College                           Importance: 0.0422
Feature Variable: % Obese                                  Importance: 0.029


In [22]:
# Feature importances using permutation importance
perm_rf = PermutationImportance(final_model_rf, random_state=1).fit(train_features, train_y)
eli5.show_weights(perm_rf, feature_names = train_features.columns.tolist())

Weight,Feature
0.1829 ± 0.0019,Mentally Unhealthy Days
0.1512 ± 0.0049,% Children in Poverty
0.0950 ± 0.0032,Teen Birth Rate
0.0685 ± 0.0017,% Some College
0.0616 ± 0.0031,% Smokers
0.0562 ± 0.0018,% Excessive Drinking
0.0433 ± 0.0018,% Physically Inactive
0.0288 ± 0.0007,% Uninsured
0.0250 ± 0.0010,% LBW
0.0249 ± 0.0009,% Mammography


## Second model I am trying is Gradient Boosting

In [23]:
## Gradient boosting regressor
GBRT = GradientBoostingRegressor(max_depth = 2, n_estimators = 1000, random_state=1)
GBRT.fit(train_features, train_y)
from sklearn.metrics import mean_squared_error
errors = [mean_squared_error(test_y, pred_y) for pred_y in GBRT.staged_predict(test_features)]
bst_n_estimators = np.argmin(errors)

GBRT_BEST = GradientBoostingRegressor(max_depth = 2, n_estimators = bst_n_estimators)
GBRT_BEST.fit(train_features, train_y)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=998, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [26]:
# evaluate model on the test set
accuracy, r2 = test_set_performance(GBRT_BEST, test_features, test_y)
print('R squared score: ', r2)
print('Accuracy:', round(accuracy, 2), '%.')

R squared score:  0.7772195989579571
1-MAPE: 90.45 %.


In [27]:
feature_importances = feature_importance_built_in(GBRT_BEST, train_features)

# Print
[print('Feature Variable: {:40} Importance: {}'.format(*pair)) for pair in feature_importances];

Feature Variable: % Smokers                                Importance: 0.1117
Feature Variable: % Excessive Drinking                     Importance: 0.1054
Feature Variable: Teen Birth Rate                          Importance: 0.096
Feature Variable: Mentally Unhealthy Days                  Importance: 0.0927
Feature Variable: % Uninsured                              Importance: 0.0822
Feature Variable: % Unemployed                             Importance: 0.0803
Feature Variable: % Some College                           Importance: 0.0695
Feature Variable: % Mammography                            Importance: 0.0643
Feature Variable: % Children in Poverty                    Importance: 0.0536
Feature Variable: Chlamydia Rate                           Importance: 0.0534
Feature Variable: % Single-Parent Households               Importance: 0.052
Feature Variable: % Physically Inactive                    Importance: 0.0485
Feature Variable: % Obese                                  Importa

## Random search for gradient boosting

In [29]:
# hyerparameters grid to search within
param_grid_gb = {'n_estimators': [100,250, 500, 750, 1000, 2000, 4000],
         'max_features': ['auto', 'sqrt'],
         'max_depth': [2, 5, 10, 20, 30],
         'min_samples_split': [5, 10,20,50,100],
         'min_samples_leaf': [1, 2, 4,10],
         'learning_rate': [0.1, 0.15, 0.2]}

# declare the regressor
GB_REGRESSOR = GradientBoostingRegressor()

# obtain the RandomizedSearchCV fitted object
random_search_gb = hyperparameter_tuning_random_search(GB_REGRESSOR, param_grid_gb, train_features, train_y)

# get the best model
final_model_gb = random_search_gb.best_estimator_

# obtain hyperparameters used in the best model
print(random_search_gb.best_params_)

# evaluate the "best" model
accuracy_final_gb, r2_final_gb = test_set_performance(final_model_gb, test_features, test_y)
print('R squared score: ', r2_final_gb)
print('1-MAPE:', round(accuracy_final_gb, 2), '%.')

{'n_estimators': 250, 'min_samples_split': 20, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'learning_rate': 0.1}
R squared score:  0.835540805833227
1-MAPE: 91.95 %.


In [30]:
# Feature importance using permutation importance
perm_gb = PermutationImportance(final_model_gb, random_state=1).fit(train_features, train_y)
eli5.show_weights(perm_gb, feature_names = train_features.columns.tolist())

Weight,Feature
0.1821 ± 0.0020,Mentally Unhealthy Days
0.1486 ± 0.0058,% Children in Poverty
0.0869 ± 0.0039,% Smokers
0.0712 ± 0.0017,Teen Birth Rate
0.0573 ± 0.0016,% Some College
0.0528 ± 0.0017,% Excessive Drinking
0.0347 ± 0.0016,% Physically Inactive
0.0318 ± 0.0004,% Uninsured
0.0314 ± 0.0012,Chlamydia Rate
0.0297 ± 0.0012,% Mammography


### Linear Regression

In [31]:
### Linear regression
from sklearn.linear_model import LinearRegression
reg = LinearRegression(normalize=True ,n_jobs = -1).fit(train_features, train_y)

accuracy, r2 = test_set_performance(reg, test_features, test_y)
print('R squared score: ', r2)
print('1-MAPE:', round(accuracy, 2), '%.')

R squared score:  0.6837854882353469
1-MAPE: 89.19 %.


In [32]:
perm_lr = PermutationImportance(reg, random_state=1).fit(train_features, train_y)
eli5.show_weights(perm_lr, feature_names = train_features.columns.tolist())

Weight,Feature
0.1973 ± 0.0071,Mentally Unhealthy Days
0.0904 ± 0.0030,% Children in Poverty
0.0310 ± 0.0031,Teen Birth Rate
0.0180 ± 0.0016,% Mammography
0.0173 ± 0.0008,% Some College
0.0116 ± 0.0015,% Excessive Drinking
0.0084 ± 0.0010,Chlamydia Rate
0.0082 ± 0.0015,% Smokers
0.0075 ± 0.0013,% LBW
0.0046 ± 0.0010,% Obese


## Summary
### 1) Random forest and gradient boosting give comparable results

In [41]:
### Performance comparisons
print('R squared score: ')
print('Random forest: ', r2_final_rf)
print('Gradient boosting: ', r2_final_gb)
print('Linear regression: ', r2)

print('1-MAPE: ')
print('Random forest: {}%'.format(round(accuracy_final_rf,2)))
print('Gradient boosting: {}%'.format(round(accuracy_final_gb,2)))
print('Linear regression: {}%'.format(round(accuracy,2)))

R squared score: 
Random forest:  0.8172780871807593
Gradient boosting:  0.835540805833227
Linear regression:  0.6837854882353469
1-MAPE: 
Random forest: 91.89%
Gradient boosting: 91.95%
Linear regression: 89.19%


### 2) Feature importances ranked by permutation importance show that "Mentally Unhealthy Days" has the biggest impact on predictions, "% Children in Poverty" is the second. Our target variable is "% reporting fair/poor health", could this be most affected by population's mental state?

## Going forward
### 1) Interested in how food environment affects people's health condition? Have county level data for grocery stores, convenience store, fast-food restuarant, full-service restaurant, soda sales tax, but only in 2014. Preliminary processing in postgresql.
### 2) Improve regression results. Standardize data? Transform data to look more like a normal distribution? Did I have effective features and did I choose the right target variable to predict?